<a href="https://colab.research.google.com/github/ggximenez/Princing-Report-B3/blob/main/pricing_reports_b3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook para extração automática de dados dos Pricing Reports da B3
####Este notebook tem a capacidade de baixar os arquivos disponíveis na B3 sobre os boletins diários que contém os dados de negociação diários de todos os ativos listados na B3, processá-los, e gerar um DataFrame Pandas ao fim.

####Os Pricing Reports contém informações sobre os mercados a vista, futuro, e de opções/a termo. Contém diversas informações úteis para análises financeiras dos ativos listados na B3 - é um resumo diário de cada um deles. Para mais informações sobre o arquivo, acesse: https://www.b3.com.br/data/files/16/70/29/9C/6219D710C8F297D7AC094EA8/Catalogo_precos_v1.3.pdf

####Com o DataFrame Pandas que resulta da execução do código é possível realizar diversas manipulações e filtragem de dados, em especial usando o método query().

In [ ]:
import subprocess
import sys
import xml.etree.ElementTree as ET

def install_and_import(package, as_=None):
    try:
        module = __import__(package)
        if as_ is not None:
            globals()[as_] = module
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        module = __import__(package)
        if as_ is not None:
            globals()[as_] = module

# Use a função para importar os módulos necessários com os respectivos alias
modules = [('os', 'os'), ('zipfile', 'zipfile'), ('pandas', 'pd'), ('numpy', 'np'),
           ('re', 're'), ('requests', 'requests'), ('glob', 'glob')]

for module, alias in modules:
    install_and_import(module, alias)


####Caso queira modificar o intervalo, modifique as datas das variáveis "inicio" e "fim". Contudo, quanto maior o intervalo maior será o tempo para a execução total da rotina.

In [ ]:
# Define o intervalo de datas
inicio = '2023-07-01'
fim = '2023-07-14'

# Cria a série de datas
datas = pd.date_range(start=inicio, end=fim, freq='B')

# Formata as datas para o padrão de nome de arquivo desejado e salve em uma lista
nomes_arquivos = [data.strftime('PR%y%m%d') for data in datas]
nomes_arquivos2 = [x+'.zip' for x in nomes_arquivos]

# Diretório onde os arquivos serão salvos
diretorio = 'downloads'
if not os.path.exists(diretorio):
    # Se não existir, cria o diretório
    os.mkdir(diretorio)

for i in nomes_arquivos2:
    url = "https://www.b3.com.br/pesquisapregao/download"

    querystring = {"filelist":f"{i},"}

    response = requests.request("GET", url, params=querystring)

    # Salva o arquivo no diretório
    arquivo = os.path.join(diretorio, i)
    with open(arquivo, "wb") as f:
        f.write(response.content)

    # Se o tamanho do arquivo for menor do que 5KB, o remove
    if os.path.getsize(arquivo) < 5 * 1024:  # 5KB são 5 * 1024 bytes
        os.remove(arquivo)

In [ ]:
def extract_zip(file_path, extract_to):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Diretório dos arquivos ZIP iniciais
start_directory = diretorio

# Diretório para colocar todos os arquivos XML
end_directory = "xmls"
if not os.path.exists(end_directory):
    # Se não existir, cria o diretório
    os.mkdir(end_directory)

for foldername, subfolders, filenames in os.walk(start_directory):
    for filename in filenames:
        if filename.endswith('.zip'):
            print(f"Extraindo {filename}...")
            extract_zip(os.path.join(foldername, filename), end_directory)

            # Extrai os arquivos ZIP internos
            for inner_foldername, inner_subfolders, inner_filenames in os.walk(end_directory):
                for inner_filename in inner_filenames:
                    if inner_filename.endswith('.zip'):
                        print(f"Extraindo {inner_filename}...")
                        extract_zip(os.path.join(inner_foldername, inner_filename), end_directory)

zip_files = glob.glob(end_directory+'/*.zip')

# Remove cada arquivo .zip encontrado
for zip_file in zip_files:
    os.remove(zip_file)

Extraindo PR230707.zip...
Extraindo PR230707.zip...
Extraindo PR230705.zip...
Extraindo PR230707.zip...
Extraindo PR230705.zip...
Extraindo PR230703.zip...
Extraindo PR230707.zip...
Extraindo PR230705.zip...
Extraindo PR230703.zip...
Extraindo PR230710.zip...
Extraindo PR230707.zip...
Extraindo PR230705.zip...
Extraindo PR230703.zip...
Extraindo PR230710.zip...
Extraindo PR230712.zip...
Extraindo PR230707.zip...
Extraindo PR230705.zip...
Extraindo PR230703.zip...
Extraindo PR230710.zip...
Extraindo PR230712.zip...
Extraindo PR230706.zip...
Extraindo PR230707.zip...
Extraindo PR230705.zip...
Extraindo PR230703.zip...
Extraindo PR230710.zip...
Extraindo PR230712.zip...
Extraindo PR230706.zip...
Extraindo PR230711.zip...
Extraindo PR230707.zip...
Extraindo PR230705.zip...
Extraindo PR230703.zip...
Extraindo PR230710.zip...
Extraindo PR230712.zip...
Extraindo PR230706.zip...
Extraindo PR230711.zip...
Extraindo PR230714.zip...
Extraindo PR230707.zip...
Extraindo PR230705.zip...
Extraindo PR

In [ ]:
diretorio = end_directory

# Dicionário para rastrear o maior sufixo para cada data
arquivos_por_data = {}

# Lista para rastrear todos os arquivos
todos_arquivos = []

# Regex para dividir o nome do arquivo em prefixo, data e sufixo
regex = re.compile(r'(BVBG\.086\.01_BV000328)(\d{8})(\d+)')

# Itere sobre todos os arquivos na pasta
for nome_arquivo in os.listdir(diretorio):

    # Use regex para dividir o nome do arquivo em prefixo, data e sufixo
    match = regex.match(nome_arquivo)
    if match:
        prefixo, data, sufixo = match.groups()
        sufixo = int(sufixo)  # Converta o sufixo para int para comparação numérica

        # Adicione o nome do arquivo à lista de todos os arquivos
        todos_arquivos.append(nome_arquivo)

        # Se a data não está no dicionário, ou se o sufixo é maior do que o atualmente rastreado, atualize o dicionário
        if data not in arquivos_por_data or sufixo > arquivos_por_data[data][1]:
            arquivos_por_data[data] = (nome_arquivo, sufixo)

# Remova os arquivos com o maior sufixo da lista de todos os arquivos
for data, (nome_arquivo, sufixo) in arquivos_por_data.items():
    todos_arquivos.remove(nome_arquivo)

# Agora, 'todos_arquivos' contém apenas os arquivos que devem ser removidos
for nome_arquivo in todos_arquivos:
    os.remove(os.path.join(diretorio, nome_arquivo))

In [ ]:
#Função que processa todos os arquivos .xml
def processar_arquivo(caminho_arquivo):
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()

    #Coleta as quantidades em nódulo TradQty
    TradQty = []
    for i in range(1, len(root[0][0])):
        try:
            TradQty.append(root[0][0][i][1][0][3][0].text)
        except IndexError:
            TradQty.append(np.nan)

    #Coleta dados em nódulo PlcOfListg
    PlcOfListg_tag = []
    for i in range(1, len(root[0][0])):
        l = []
        for j in range(0, len(root[0][0][i][1][0][2][1])):
            l.append(root[0][0][i][1][0][2][1][j].tag)
        PlcOfListg_tag.append(l)

    #Coleta dados em nódulo PlcOfListg
    PlcOfListg_tag_cl = [[re.sub(r'\{.*?\}', '', item) for item in sub_lista] for sub_lista in PlcOfListg_tag]

    #Coleta dados em nódulo PlcOfListg
    PlcOfListg_text = []
    for i in range(1, len(root[0][0])):
        l = []
        for j in range(0, len(root[0][0][i][1][0][2][1])):
            l.append(root[0][0][i][1][0][2][1][j].text)
        PlcOfListg_text.append(l)

    #Cria lista de dicionários com dados do nódulo PlcOfListg
    PlcOfListg = [dict(zip(t, v)) for t, v in zip(PlcOfListg_tag_cl, PlcOfListg_text)]

    #Coleta dados em nódulo OthrId
    OthrId_tag = []
    for i in range(1, len(root[0][0])):
        l = []
        for j in range(0, len(root[0][0][i][1][0][2][0])):
            l.append(root[0][0][i][1][0][2][0][j].tag)
        OthrId_tag.append(l)

    #Coleta dados em nódulo OthrId
    OthrId_tag_cl = [[re.sub(r'\{.*?\}', '', item) for item in sub_lista] for sub_lista in OthrId_tag]

    #Coleta dados em nódulo OthrId
    OthrId_text = []
    for i in range(1, len(root[0][0])):
        l = []
        for j in range(0, len(root[0][0][i][1][0][2][0])):
            l.append(root[0][0][i][1][0][2][0][j].text)
        OthrId_text.append(l)

    #Cria lista de dicionários com dados do nódulo OthrId
    OthrId = [dict(zip(t, v)) for t, v in zip(OthrId_tag_cl, OthrId_text)]

    #Coleta os Tickers
    ticker = []
    for i in range(1, len(root[0][0])):
        ticker.append(root[0][0][i][1][0][1][0].text)

    #Coleta as datas
    dt = []
    for i in range(1, len(root[0][0])):
        dt.append(root[0][0][i][1][0][0][0].text)

    #Coleta dados em nódulo FinInstrmAttrbts
    FinInstrmAttrbts_tags = []
    for i in range(1, len(root[0][0])):
        tags_i = []
        for j in range(0, len(root[0][0][i][1][0][4])):
            tags_i.append(root[0][0][i][1][0][4][j].tag)
        FinInstrmAttrbts_tags.append(tags_i)

    #Coleta dados em nódulo FinInstrmAttrbts
    FinInstrmAttrbts_tags_cl = [[re.sub(r'\{.*?\}', '', item) for item in sub_lista] for sub_lista in FinInstrmAttrbts_tags]

    #Coleta dados em nódulo FinInstrmAttrbts
    FinInstrmAttrbts_text = []
    for i in range(1, len(root[0][0])):
        tags_i = []
        for j in range(0, len(root[0][0][i][1][0][4])):
            tags_i.append(root[0][0][i][1][0][4][j].text)
        FinInstrmAttrbts_text.append(tags_i)

    #Cria lista de dicionários com dados em nódulo FinInstrmAttrbts
    FinInstrmAttrbts = [dict(zip(t, v)) for t, v in zip(FinInstrmAttrbts_tags_cl, FinInstrmAttrbts_text)]

    #Une as listas de dicionários
    for dic, tk in zip(FinInstrmAttrbts, ticker):
        dic["TckrSymb"] = tk
    for dic, data in zip(FinInstrmAttrbts, dt):
        dic["Dt"] = data
    for dic, tdqt in zip(FinInstrmAttrbts, TradQty):
        dic["TradQty"] = tdqt

    listas_combinadas = [{**dic1, **dic2, **dic3} for dic1, dic2, dic3 in zip(FinInstrmAttrbts, OthrId, PlcOfListg)]

    #Dicionário com chaves inteiras
    legendas = {"TradDt": "TradeDate",
                "Dt": "Date",
                "SctyId": "SecurityIdentification",
                "TckrSymb": "TickerSymbol",
                "FinInstrmId": "FinancialInstrumentIdentification",
                "OthrId": "OtherIdentification",
                "Id": "Identification",
                "Tp": "Type",
                "Prtry": "Proprietary",
                "PlcOfListg": "PlaceOfListing",
                "MktIdrCd": "MarketIdentifierCode",
                "TradDtls": "TradeDetails",
                "DaysToSttlm": "DaysToSettlement",
                "TradQty": "TradeQuantity",
                "FinInstrmAttrbts": "FinancialInstrumentAttributes",
                "MktDataStrmId": "MarketDataStreamIdentification",
                "NtlFinVol": "NationalFinancialVolume",
                "IntlFinVol": "InternationalFinancialVolume",
                "OpnIntrst": "OpenInterest",
                "FinInstrmQty": "FinancialInstrumentQuantity",
                "BestBidPric": "BestBidPrice",
                "BestAskPric": "BestAskPrice",
                "FrstPric": "FirstPrice",
                "MinPric": "MinimumPrice",
                "MaxPric": "MaximumPrice",
                "TradAvrgPric": "TradeAveragePrice",
                "LastPric": "LastPrice",
                "RglrTxsQty": "RegularTransactionsQuantity",
                "NonRglrTxsQty": "NonRegularTransactionsQuantity",
                "RglrTraddCtrcts": "RegularTradedContracts",
                "NonRglrTraddCtrcts": "NonRegularTradedContracts",
                "NtlRglrVol": "NationalRegularVolume",
                "NtlNonRglrVol": "NationalNonRegularVolume",
                "IntlRglrVol": "InternationalRegularVolume",
                "IntlNonRglrVol": "InternationalNonRegularVolume",
                "AdjstdQt": "AdjustedQuote",
                "AdjstdQtTax": "AdjustedQuoteTax",
                "AdjstdQtStin": "AdjustedQuoteSituation",
                "PrvsAdjstdQt": "PreviousAdjustedQuote",
                "PrvsAdjstdQtTax": "PreviousAdjustedQuoteTax",
                "PrvsAdjstdQtStin": "PreviousAdjustedQuoteSituation",
                "OscnPctg": "OscillationPercentage",
                "VartnPts": "VariationPoints",
                "EqvtVal": "EquivalentValue",
                "AdjstdValCtrct": "AdjustedValueContract",
                "MaxTradLmt": "MaximumTradeLimit",
                "MinTradLmt": "MinimumTradeLimit"}

    #Lista de dicionários final
    lista_final = [{legendas.get(k, k): v for k, v in d.items()} for d in listas_combinadas]

    #Converte lista de dicionários para DataFrame pandas
    df = pd.DataFrame(lista_final)

    return df

In [ ]:
#Função para armazenar o dataframe enquanto concatena
df_total = pd.DataFrame()

In [ ]:
#Função que concatena todos os dataframes:
for nome_arquivo in os.listdir(diretorio):
    # Verifique se é um arquivo XML
    if nome_arquivo.endswith('.xml'):
        # Junte o caminho do diretório com o nome do arquivo
        caminho_arquivo = os.path.join(diretorio, nome_arquivo)
        # Processe o arquivo e obtenha o DataFrame
        df = processar_arquivo(caminho_arquivo)
        # Adicione os dados ao DataFrame total
        df_total = pd.concat([df_total, df])

In [ ]:
df_total

,MarketDataStreamIdentification,NationalFinancialVolume,InternationalFinancialVolume,FinancialInstrumentQuantity,BestBidPrice,BestAskPrice,FirstPrice,MinimumPrice,MaximumPrice,TradeAveragePrice,...,OpenInterest,EquivalentValue,AdjustedQuote,AdjustedQuoteTax,AdjustedQuoteSituation,PreviousAdjustedQuote,PreviousAdjustedQuoteTax,PreviousAdjustedQuoteSituation,VariationPoints,AdjustedValueContract
0,E,369444.55,77020.56,17620,20.52,20.65,20.9,20.52,21.45,20.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E,70,14.59,100,NaN,NaN,0.7,0.7,0.7,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34050,E,5322991.5,1094680.1,210,56.15,56.21,56.4,56.06,56.59,56.32,...,52253,NaN,56.27,NaN,F,55.89,NaN,F,0.38,171
34051,E,2959267.5,608577.2,110,59.57,59.8,59.81,59.53,60.06,59.78,...,21498,NaN,59.78,NaN,F,59.46,NaN,F,0.32,144
34052,E,234514575,47909982.83,955,NaN,NaN,16.4,16.4,16.4,16.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34053,E,272937.73,56130,3,186.7,187,187.7,186.65,187.7,187.1,...,5583,907.6,186.65,NaN,F,190.8,NaN,F,-4.15,-2017.979
